In [1]:
import pandas as pd
import numpy as np
from google.colab import files
from datetime import timedelta

In [2]:
uploaded = files.upload()
dataset = pd.read_csv("dataset.csv").head(260000)


Saving dataset.csv to dataset.csv


<ipython-input-2-4272e35d6b4b>:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("dataset.csv").head(260000)


In [3]:
def anonimize_train_number(train_number_col):
    # Извлечение числовой части из строк
    numeric_values = train_number_col.str.extract('(\d+)')[0].astype(int)

    # Вычисляем среднее значение
    mean_value = numeric_values.mean()

    def transform_value(x):
        number = int(x)  # Преобразуем в целое число
        if number < mean_value:
            # Генерируем случайное число от 0 до number и от number до mean_value
            random_first = np.random.randint(0, number + 1)
            random_second = np.random.randint(number, int(mean_value) + 1)
        else:
            # Генерируем случайное число от 0 до mean_value и от mean_value до number
            random_first = np.random.randint(0, int(mean_value) + 1)
            random_second = np.random.randint(int(mean_value), number + 1)

        return f"{random_first}-{random_second}"

    # Применяем трансформацию ко всем значениям
    return train_number_col.apply(lambda x: transform_value(x[:-1]))

def anonimize_wagon_seat(wagon_seat_col):
    # Функция для анонимизации одного значения
    def anonymize_seat(seat):
        # Разделяем номер вагона и номер места
        wagon, seat_number = map(int, seat.split('-'))

        # Генерируем случайные значения в пределах диапазона
        random_seat_number = np.random.randint(1, seat_number + 1)

        return f"{wagon}-{random_seat_number}"

    # Применяем анонимизацию ко всем значениям в колонке
    return wagon_seat_col.apply(anonymize_seat)

def anonymize_price(price_col, percentage=0.3):
    # Очистка данных: удаляем ' руб' и преобразуем в float
    price_col = price_col.str.replace(' руб', '', regex=False).astype(float)

    # Анонимизация цен с использованием случайных значений в пределах указанного процента
    return price_col.apply(lambda x: np.random.uniform(x * (1 - percentage), x * (1 + percentage)))


# Функция для анонимизации временных меток
def anonymize_timestamps(timestamp_col):
    # Преобразуем временные метки в datetime один раз
    timestamps = pd.to_datetime(timestamp_col)

    # Генерируем случайные смещения в диапазоне от -48 до 48 часов
    offsets = np.random.randint(-48, 49, size=len(timestamps))

    # Применяем смещения ко всем временным меткам
    return timestamps + pd.to_timedelta(offsets, unit='h')

In [4]:
def calculate_k_anonymity(dataset, attributes):
    # Группируем данные по выбранным атрибутам и подсчитываем количество записей в каждой группе
    group_counts = dataset.groupby(attributes).size().reset_index(name='count')

    # Находим минимальное количество записей в группе
    min_count = group_counts['count'].min()

    return min_count  # Возвращаем минимальный размер группы

In [5]:
def anonimize_dataset(dataset):
  print(dataset.info())
  print(dataset.head())

  # g) Маскеризация
  dataset["pasport_number"] = dataset['pasport_number'].apply(lambda x: f"XXXX XX{x[-2:]}")
  # dataset["card_number"] = dataset['card_number'].apply(lambda x: f"XXXX XXXX XXXX {str(int(x))[-4:]}" if pd.notna(x) else "XXXX XXXX XXXX XXXX")

  # Кастомный +  g)Маскеризация
  dataset['train_number'] = anonimize_train_number(dataset['train_number'])

  # f)Создание псевдонимов
  dataset['direction'] = dataset[["from_station", "to_station"]].agg(" - ".join, axis=1)
  dataset['wagon_seat'] = anonimize_wagon_seat(dataset['wagon_seat'])


  # c)Возмущение
  dataset['price'] = anonymize_price(dataset['price'])
  dataset['departure'] = anonymize_timestamps(dataset['departure'])
  dataset['arrival'] = anonymize_timestamps(dataset['arrival'])


  # i)Удаление атрибутов
  dataset.drop(columns=["Unnamed: 0", "full_name", "from_station", "to_station", "direction"], axis=1, inplace=True)


  # e)Перемешивание
  dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

  return dataset


In [6]:
anonim_dataset = anonimize_dataset(dataset.copy())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260000 entries, 0 to 259999
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      260000 non-null  int64 
 1   full_name       260000 non-null  object
 2   pasport_number  260000 non-null  object
 3   train_number    260000 non-null  object
 4   from_station    260000 non-null  object
 5   to_station      260000 non-null  object
 6   wagon_seat      260000 non-null  object
 7   price           260000 non-null  object
 8   departure       260000 non-null  object
 9   arrival         260000 non-null  object
dtypes: int64(1), object(9)
memory usage: 19.8+ MB
None
   Unnamed: 0                      full_name pasport_number train_number  \
0           0  Онуфрий Бенедиктович Молчанов    7840 472337         028С   
1           1         Майя Аркадьевна Жукова    5696 409232         068С   
2           2     Артемьев Олимпий Архипович    0754 595967      

In [ ]:
# Адекватная анонимизация
k = 3
attributes = ['pasport_number', 'card_number', 'train_number', 'wagon_seat']  # Выбор атрибутов для анализа

print(anonim_dataset.groupby(attributes).size())

min_count = calculate_k_anonymity(anonim_dataset, attributes)

print(f"Минимальное количество записей в группе: {min_count}")
print(f"Данные {'достигли' if min_count >= k else 'не достигли'} {k}-анонимности.")



K-анонимность обезличенного набора: 1

Плохие значения K-анонимности:
K = 1, процент = 100.00% (повторяется 259996 раз)
K = 2, процент = 0.00% (повторяется 2 раз)

Уникальные строки при K=1:
       pasport_number          card_number train_number wagon_seat  \
0           XXXX XX68  XXXX XXXX XXXX 8788        24-53       1-11   
1           XXXX XX70  XXXX XXXX XXXX 3674         2-62        1-7   
2           XXXX XX47  XXXX XXXX XXXX 5807        13-45        1-6   
3           XXXX XX73  XXXX XXXX XXXX 2094        13-19       1-13   
4           XXXX XX41  XXXX XXXX XXXX 3739         4-84        1-3   
...               ...                  ...          ...        ...   
259995      XXXX XX84  XXXX XXXX XXXX 9874        15-64        1-7   
259996      XXXX XX68  XXXX XXXX XXXX 0916         9-53        1-8   
259997      XXXX XX79  XXXX XXXX XXXX 2876        10-29        1-8   
259998      XXXX XX58  XXXX XXXX XXXX 4301        16-78        1-4   
259999      XXXX XX38  XXXX XXXX XXXX 

In [9]:
# анонимизация для лабы

import pandas as pd
import numpy as np
from datetime import timedelta

def anonymize_full_name(df):
    # Обобщение полного имени по первой букве
    df['full_name'] = df['full_name'].apply(lambda x: f"{x[0]}XXXX XXXX")
    return df

def anonymize_passport_number(df):
    # Обобщение номера паспорта, скрываем часть
    df['pasport_number'] = df['pasport_number'].apply(lambda x: f"XXXXXX{x[-1:]}")
    return df

def anonymize_card_number(df):
    # Маскирование номера карты
    df['card_number'] = df['card_number'].apply(lambda x: f"XXXX XXXX XXXX {int(x) % 10000}")
    return df

def anonymize_train_number(train_number_col):
    # Обобщение для номеров поездов (например, группируем по первой цифре)
    return train_number_col.apply(lambda x: f"{x[0]}XX")

def anonymize_price(price_col, percentage=0.3):
    # Возмущение цен с предварительным удалением символов валюты
    return price_col.apply(lambda x: float(x.split()[0]) * np.random.uniform(1 - percentage, 1 + percentage))


def anonymize_timestamps(timestamp_col):
    # Обезличивание временных меток
    timestamps = pd.to_datetime(timestamp_col)
    offsets = np.random.randint(-48, 49, size=len(timestamps))
    return timestamps + pd.to_timedelta(offsets, unit='h')

def calculate_k_anonymity(dataset, attributes):
    # Группируем данные по выбранным атрибутам и подсчитываем количество записей в каждой группе
    group_counts = dataset.groupby(attributes).size().reset_index(name='count')
    min_count = group_counts['count'].min()
    return min_count, len(group_counts)

def anonymize_dataset(dataset, quasi_identifiers):
    # a) Обезличивание входного датасета
    dataset = anonymize_full_name(dataset)
    dataset = anonymize_passport_number(dataset)
    # dataset = anonymize_card_number(dataset)
    dataset['train_number'] = anonymize_train_number(dataset['train_number'])
    dataset['price'] = anonymize_price(dataset['price'])
    dataset['departure'] = anonymize_timestamps(dataset['departure'])
    dataset['arrival'] = anonymize_timestamps(dataset['arrival'])

    # Удаление не нужных атрибутов
    dataset.drop(columns=["Unnamed: 0", "from_station", "to_station"], inplace=True)

    # Перемешивание
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

def main(dataset, quasi_identifiers):
    # Шаг 1: Анонимизация датасета
    anonymized_data = anonymize_dataset(dataset, quasi_identifiers)

    # Шаг 2: Вычисление K-анонимити
    k_value, total_groups = calculate_k_anonymity(anonymized_data, quasi_identifiers)

    # Шаг 3: Вывод значений K-анонимити
    print(f"K-анонимити: {k_value}")

    # Шаг 4: Плохие значения K-анонимити
    bad_k_values = anonymized_data.groupby(quasi_identifiers).size().reset_index(name='count')
    bad_k_values = bad_k_values[bad_k_values['count'] < 7]  # Изменено на 7

    if not bad_k_values.empty:
        print("Плохие значения K-анонимити:")
        print(bad_k_values)

    # Шаг 5: Количество уникальных строк по квази-идентификаторам
    unique_rows = anonymized_data[quasi_identifiers].drop_duplicates()
    print(f"Количество уникальных строк: {len(unique_rows)}")

    if k_value == 1:
        print("Уникальные строки с K=1:")
        print(unique_rows)

    # Шаг 6: Оценка приемлемого значения K
    record_count = len(dataset)
    if record_count <= 51000:
        acceptable_k = 10
    elif record_count <= 105000:
        acceptable_k = 7
    else:
        acceptable_k = 5

    print(f"Приемлемое значение K: {acceptable_k}")
    print("Обнаруженное K-анонимити:", k_value)

    # Шаг 7: Полезность данных
    original_size = dataset.memory_usage(deep=True).sum()
    anonymized_size = anonymized_data.memory_usage(deep=True).sum()
    size_difference = original_size - anonymized_size

    print(f"Размер исходного набора: {original_size} байт")
    print(f"Размер обезличенного набора: {anonymized_size} байт")
    print(f"Сокращение размера: {size_difference} байт")


quasi_identifiers = ['train_number', 'departure']  # Укажите ваши квази-идентификаторы
main(dataset.copy(), quasi_identifiers)


Число записей в исходном наборе: 260000
K-анонимность обезличенного набора: 578

Похожие значения K-анонимности:
K = 1, процент = 0.00% (повторяется 0 раз)
K = 2, процент = 0.00% (повторяется 0 раз)
K = 3, процент = 0.00% (повторяется 0 раз)
K = 4, процент = 0.00% (повторяется 0 раз)
K = 5, процент = 0.00% (повторяется 0 раз)
K = 6, процент = 0.00% (повторяется 0 раз)
K = 7, процент = 0.00% (повторяется 0 раз)

Приемлемое значение K-анонимности для набора данных: K >= 7
Обезличенный датасет сохранён в 'anonymized_dataset.xml'.
